In [1]:
import pandas as pd

def _url_to_img_tag(path):
    return f"""<img
    style="width: 100px; height: 100px; object-fit: contain;"
            src="{path}"/>"""


def styled(df: pd.DataFrame) -> pd.DataFrame:
    return df.style.format({'main_image': _url_to_img_tag,
                            'image_url': _url_to_img_tag,
                            'image_url_lhs': _url_to_img_tag,
                            'image_url_rhs': _url_to_img_tag})

In [2]:
from elasticsearch import Elasticsearch 

search_template_src = """
{{!
  This is an Elasticsearch query expressed as a Mustache template. Its uploaded as a 'search template' and
  Elasticsearch uses it as kind of stored query that can be executed with different parameters. It allows
  us to 'program' Elasticsearch with query logic we can test independent of the query-side application (ie liaison)

  See https://www.elastic.co/guide/en/elasticsearch/reference/current/search-template.html
}}
{
  "query": {
    "bool": {
      {{! lexical scoring components - showing candidates with All matches }}
      "should": [
        {"bool": {
          "filter": [
            {"bool": {
              "filter": [
                {"range": {"min_price": {"gte": {{min_price}}}}}
              ]}},
            {"bool": {
              "filter": [
                {"range": {"max_price": {"lte": {{max_price}}}}}
              ]}},
            {"bool": {
              "should": [
                {{#categories}}
                {"match_phrase": {"category": "{{value}}"}} {{^last}},{{/last}}
                {{/categories}}
               ]}},
              {"bool": {
              "should": [
                {{#brand_aesthetics}}
                {"match_phrase": {"brand_name.aesthetic": "{{value}}"}} {{^last}},{{/last}}
                {{/brand_aesthetics}}
              ]}},
             {"bool": {
              "should": [
                {{#brand_ages}}
                {"match_phrase": {"brand_name.age": "{{value}}"}} {{^last}},{{/last}}
                {{/brand_ages}}
              ]}},
            {"bool": {
              "should": [
                {{#brand_occasions}}
                {"match_phrase": {"brand_name.occasion": "{{value}}"}} {{^last}},{{/last}}
                {{/brand_occasions}}
              ]}},
            {"bool": {
              "should": [
                {{#brand_prices}}
                {"match_phrase": {"brand_name.price": "{{value}}"}} {{^last}},{{/last}}
                {{/brand_prices}}
              ]}},
            {"bool": {
              "filter": [
                {{#departments}}
                {"match": {"department": "{{value}}"}} {{^last}},{{/last}}
                {{/departments}}
              ]}},
            {"bool": {
              "filter": [
                {{#brands}}
                {"match": {"brand_name.en": "{{value}}"}} {{^last}},{{/last}}
                {{/brands}}
              ]}},
            {"bool": {
              "should": [
                {{#sizes}}
                {"match": {"standard_sizes": "{{value}}"}} {{^last}},{{/last}}
                {{/sizes}}
              ]}}

            {{#has_option_ids}}
            ,{"ids":
               {"values": [
               {{#option_ids}}
               "{{value}}" {{^last}},{{/last}}
               {{/option_ids}}
             ]}}
            {{/has_option_ids}}
          ],
          "must": [
            {"multi_match": {
              "query": "{{keywords}}",
              "fields": ["name.en", "description.en", "brand_name.en"],
              "type": "cross_fields",
              "minimum_should_match": "100%"
            }}
          ]
        }}
      {{! image vector component - high precision matches, boosted higher }}
        {{#image_vector}}
        ,{"knn": {
          "field": "image_embedding",
          "boost": 1000,
          "similarity": 0.2,
          "k": 10,
          "filter": {
            {{! annoying - repeated lexical filters for prefiltering before vector search}}
            "bool": {
              "filter": [
                {"multi_match": {
                  "query": "{{keywords}}",
                  "fields": ["name.en", "description.en", "brand_name.en"],
                  "type": "cross_fields",
                  "minimum_should_match": "100%"
                }},
                {"bool": {
                  "filter": [
                    {"range": {"min_price": {"gte": {{min_price}}}}}
                  ]}},
                {"bool": {
                  "filter": [
                    {"range": {"max_price": {"lte": {{max_price}}}}}
                  ]}},
                {"bool": {
                  "should": [
                    {{#brand_aesthetics}}
                    {"match_phrase": {"brand_name.aesthetic": "{{value}}"}} {{^last}},{{/last}}
                    {{/brand_aesthetics}}
                  ]}},
                 {"bool": {
                  "should": [
                    {{#brand_ages}}
                    {"match_phrase": {"brand_name.age": "{{value}}"}} {{^last}},{{/last}}
                    {{/brand_ages}}
                  ]}},
                {"bool": {
                  "should": [
                    {{#brand_occasions}}
                    {"match_phrase": {"brand_name.occasion": "{{value}}"}} {{^last}},{{/last}}
                    {{/brand_occasions}}
                  ]}},
                {"bool": {
                  "should": [
                    {{#brand_prices}}
                    {"match_phrase": {"brand_name.price": "{{value}}"}} {{^last}},{{/last}}
                    {{/brand_prices}}
                  ]}},
                {"bool": {
                  "filter": [
                    {{#departments}}
                    {"match": {"department": "{{value}}"}} {{^last}},{{/last}}
                    {{/departments}}
                  ]}},
                {"bool": {
                  "filter": [
                    {{#brands}}
                    {"match": {"brand_name.en": "{{value}}"}} {{^last}},{{/last}}
                    {{/brands}}
                  ]}},
                  {"bool": {
                    "should": [
                      {{#sizes}}
                      {"match": {"standard_sizes": "{{value}}"}} {{^last}},{{/last}}
                      {{/sizes}}
                    ]}}
                 {{#has_option_ids}}
                 ,{"ids":
                    {"values": [
                    {{#option_ids}}
                    "{{value}}" {{^last}},{{/last}}
                    {{/option_ids}}
                  ]}}
                 {{/has_option_ids}}
              ]
            }
          },
          "query_vector": {{image_vector}}
        }}
        {{/image_vector}}
      {{! image vector component - lower precision matches }}
        {{#image_vector}}
        ,{"knn": {
          "field": "image_embedding",
          "boost": 100,
          "similarity": 0.2,
          "k": 10,
          "filter": {
            {{! repeated lexical filters for prefiltering before vector search}}
            "bool": {
              "filter": [
                {"multi_match": {
                  "query": "{{keywords}}",
                  "fields": ["name.en", "description.en", "brand_name.en"],
                  "type": "cross_fields",
                  "minimum_should_match": "1"
                }},
                {"bool": {
                  "filter": [
                    {"range": {"min_price": {"gte": {{min_price}}}}}
                  ]}},
                {"bool": {
                  "filter": [
                    {"range": {"max_price": {"lte": {{max_price}}}}}
                  ]}},
                {"bool": {
                  "should": [
                    {{#categories}}
                    {"match_phrase": {"category": "{{value}}"}} {{^last}},{{/last}}
                    {{/categories}}
                  ]}},
                 {"bool": {
                  "should": [
                    {{#brand_aesthetics}}
                    {"match_phrase": {"brand_name.aesthetic": "{{value}}"}} {{^last}},{{/last}}
                    {{/brand_aesthetics}}
                  ]}},
                 {"bool": {
                  "should": [
                    {{#brand_ages}}
                    {"match_phrase": {"brand_name.age": "{{value}}"}} {{^last}},{{/last}}
                    {{/brand_ages}}
                  ]}},
                {"bool": {
                  "should": [
                    {{#brand_occasions}}
                    {"match_phrase": {"brand_name.occasion": "{{value}}"}} {{^last}},{{/last}}
                    {{/brand_occasions}}
                  ]}},
                {"bool": {
                  "should": [
                    {{#brand_prices}}
                    {"match_phrase": {"brand_name.price": "{{value}}"}} {{^last}},{{/last}}
                    {{/brand_prices}}
                  ]}},
                {"bool": {
                  "filter": [
                    {{#departments}}
                    {"match": {"department": "{{value}}"}} {{^last}},{{/last}}
                    {{/departments}}
                  ]}},
                {"bool": {
                  "filter": [
                    {{#brands}}
                    {"match": {"brand_name.en": "{{value}}"}} {{^last}},{{/last}}
                    {{/brands}}
                  ]}},
                  {"bool": {
                    "should": [
                      {{#sizes}}
                      {"match": {"standard_sizes": "{{value}}"}} {{^last}},{{/last}}
                      {{/sizes}}
                    ]}}
                 {{#has_option_ids}}
                 ,{"ids":
                    {"values": [
                    {{#option_ids}}
                    "{{value}}" {{^last}},{{/last}}
                    {{/option_ids}}
                  ]}}
                 {{/has_option_ids}}
              ]
            }
          },
          "query_vector": {{image_vector}}
        }}
        {{/image_vector}}
     ]
}}}
"""
es = Elasticsearch("http://localhost:9200")
script_spec = {"script": {"lang": "mustache", "source": search_template_src}}
es.put_script(body=script_spec, id="my-search-template")

ObjectApiResponse({'acknowledged': True})

In [3]:
import json

template_params = {
    "params": {
        "keywords": "search keywords",
        "image_vector": "[1,2,3]",
        "min_price": 0,
        "brand_prices": [{"value": "fancy", "last": True}],
        "max_price": 100000000,
        "categories": [{"value": "shirt", "last": True}]
    }
}
templ_out = es.render_search_template(id="my-search-template", body=template_params)['template_output']

print(json.dumps(templ_out, indent=2))

{
  "query": {
    "bool": {
      "should": [
        {
          "bool": {
            "filter": [
              {
                "bool": {
                  "filter": [
                    {
                      "range": {
                        "min_price": {
                          "gte": 0
                        }
                      }
                    }
                  ]
                }
              },
              {
                "bool": {
                  "filter": [
                    {
                      "range": {
                        "max_price": {
                          "lte": 100000000
                        }
                      }
                    }
                  ]
                }
              },
              {
                "bool": {
                  "should": [
                    {
                      "match_phrase": {
                        "category": "shirt"
                      }
                    }
           

In [6]:
from local_llm_judge.search_backend import local_es

df, settings = local_es.search("genz sweaters", "m")
styled(df)

,name,brand_name,id,category,main_image,main_image_path,description,score,backend
0,Geo ribbed sweater,RICK OWENS,fa5de113-56de-4b78-a8a5-44c6f9279126,TOPS,,data/img/fa5de113-56de-4b78-a8a5-44c6f9279126.png,forest green virgin wool knitted construction panelled design ribbed detailing crew neck long sleeves short side slits straight hem,795.263300,local_es
1,Wool Sweater at Nordstrom,Burberry,f9f99fb3-73bf-4326-9f61-1e63d96c2082,TOPS,,data/img/f9f99fb3-73bf-4326-9f61-1e63d96c2082.png,"An Italian-made sweater in wool jersey, cut to a regular fit. The style is embroidered with the B shield, a motif inspired by our Equestrian Knight Design.||Rib-knit trims|Embroidered B shield at chest Regular fit. Fits true to size, take your normal size. Length: 68cm/26.8in (size UK M) Model’s height: 185cm/6ft 1in. Model wears size UK M 100% wool Trim: 95% wool, 4% polyamide, 1% elastane Specialist dry clean Made in Italy",792.498050,local_es
2,wool sweater,MAISON MARGIELA,e00808f6-947b-456f-9bd0-6c93ab21a7dc,TOPS,,data/img/e00808f6-947b-456f-9bd0-6c93ab21a7dc.png,black merino wool fine knit signature Work-in-progress stitches signature four-stitch logo crew neck long sleeves ribbed cuffs and hem,791.934940,local_es
3,Essick sweater,KIKO KOSTADINOV,09acd67e-26e9-4190-b604-aa063e694597,TOPS,,data/img/09acd67e-26e9-4190-b604-aa063e694597.png,multicolour cotton knitted construction fisherman's knit horizontal stripe pattern high neck drop shoulder long raglan sleeves asymmetric hem,790.297400,local_es
4,Mohair & Wool Blend V-Neck Sweater at Nordstrom,NOON GOONS,02d4ea77-9dff-4100-99ae-1802b645d619,TOPS,,data/img/02d4ea77-9dff-4100-99ae-1802b645d619.png,"Dropped shoulder seams emphasize the boxy, relaxed fit of a lookbook-featured sweater knit from a rich mohair-and-wool yarn blend and framed by contrasting ribbed trim. 28"" length (size Medium) V-neck Long sleeves Dropped shoulders Ribbed cuffs and hem 40% nylon, 32% mohair, 28% wool Machine wash, dry flat Imported Designer Clothing",789.839700,local_es
5,Patch Crewneck Sweater - Men's - - Wool,Balenciaga,ee24aa40-ce13-420f-b8be-da4cea8461be,TOPS,,data/img/ee24aa40-ce13-420f-b8be-da4cea8461be.png,Patch Crewneck Sweater in black wool knit. Wool knit crewneck long sleeves dropped shoulders balenciaga rubber patch on the sleeve made in italy. main material: 100% wool.,789.565730,local_es
6,high-neck wool-cashmere blend sweater,POLO RALPH LAUREN,5676166f-2833-40eb-b7ca-f902d4cd7cd7,TOPS,,data/img/5676166f-2833-40eb-b7ca-f902d4cd7cd7.png,black wool-cashmere blend knitted construction high neck long sleeves ribbed cuffs and hem straight hem,786.880600,local_es
7,V-Neck Merino Wool Sweater at Nordstrom,MANGO,0f833739-3744-4671-8e60-08471eb1e5b0,TOPS,,data/img/0f833739-3744-4671-8e60-08471eb1e5b0.png,"A classic for cool weather, this finely knit merino-wool sweater features a V-neck and a regular fit for easy layering. V-neck Long sleeves 100% wool Machine wash, dry flat Imported",786.474200,local_es
8,Level turtleneck sweater,RICK OWENS,542c2d20-636b-4fbf-b614-0d7ec19b76d7,TOPS,,data/img/542c2d20-636b-4fbf-b614-0d7ec19b76d7.png,black virgin wool fine knit roll neck long sleeves ribbed cuffs and hem straight hem,786.097840,local_es
9,Le Pull Jacquard Logo Brushed Alpaca & Merino Wool Blend Sweater at Nordstrom,JACQUEMUS,0209e1bc-39bd-4233-b364-8c55672ebe1a,TOPS,,data/img/0209e1bc-39bd-4233-b364-8c55672ebe1a.png,"A brushed alpaca blend enriched with soft merino wool warms this relaxed crewneck sweater arced with a jacquard-knit logo across the back. Crewneck Long sleeves Dropped shoulders Ribbed cuffs and hem 68% alpaca, 22% polyamide, 10% merino wool Dry clean Made in Bulgaria Designer Clothing",785.640600,local_es


In [8]:
from local_llm_judge.search_backend import local_es

df, settings = local_es.search("boomer sweaters", "m")
styled(df)

,name,brand_name,id,category,main_image,main_image_path,description,score,backend
0,John John Colorblock Jacquard Wool Blend Sweater at Nordstrom,BOTTEGA VENETA,93959682-f6e1-40ef-85f0-f46ce7e2049b,TOPS,,data/img/93959682-f6e1-40ef-85f0-f46ce7e2049b.png,"Hand-embroidered red details lend a vibrant pop of color to the already bold design of this woollen crewneck sweater softened with a kiss of cashmere. 25"" length (size Large) Crewneck Long sleeves Ribbed cuffs and hem 85% wool, 9% polyamide, 5% cashmere, 1% elastane Dry clean Made in Italy Men's Designer Clothing",744.919000,local_es
1,camouflage sweater,Emporio Armani,ee659417-6204-4ded-a5ac-12051f612c89,TOPS,,data/img/ee659417-6204-4ded-a5ac-12051f612c89.png,grey knitted construction brushed effect camouflage pattern crew neck drop shoulder long sleeves ribbed cuffs and hem,740.056700,local_es
2,reindeer-patterned sweater,POLO RALPH LAUREN,99875f28-a524-4cfc-9464-2e63ddcf8a43,TOPS,,data/img/99875f28-a524-4cfc-9464-2e63ddcf8a43.png,royal blue/white/blood red wool ribbed knit reindeer-patterned jacquard crew neck drop shoulder long sleeves jacquard logo motif straight hem,739.337900,local_es
3,Logo-Embroidered Intarsia Cashmere Sweater - Men,POLO RALPH LAUREN,e1b390f1-bfd2-4f39-8cd0-018ffc87417c,TOPS,,data/img/e1b390f1-bfd2-4f39-8cd0-018ffc87417c.png,"Polo Ralph Lauren""s sweater is knitted using an intarsia technique to realise the dog-in-bobble-hat motif on the front. Perfect for festive get-togethers, it""s made from cashmere that""s carefully sourced for its quality.",738.863400,local_es
4,FLETCHER SWEATER,Trina Turk,316a3c51-5f5d-468b-aee0-666098095af9,TOPS,,data/img/316a3c51-5f5d-468b-aee0-666098095af9.png,"A fun, Boston Terrier graphic updates the classic crewneck pullover sweater adding whimsical style to your weekday or weekend look. Crewneck Ribbed neck, cuffs, and hem Long sleeves Slim fit Contrast dog silhouette design Model is 6'1"", wearing size M See Men's Size Guide FABRIC: Merino Wool 100% Merino Wool Dry clean Imported",736.482600,local_es
5,Cat Mohair Blend Crewneck Sweater at Nordstrom,GIVENCHY,dd1baa64-c41f-4d04-adc4-94c56e8291be,TOPS,,data/img/dd1baa64-c41f-4d04-adc4-94c56e8291be.png,"A shadowy cat with icy blue eyes takes center stage on a roomy sweater knit from a mohair-warmed blend. Crewneck Long sleeves Dropped shoulders Ribbed cuffs and hem 50% mohair, 40% polyamide, 8% viscose, 2% wool Dry clean Made in Italy Designer Clothing",735.981000,local_es
6,Drussellh Sweater,ISABEL MARANT,f2244eb6-9735-4e0b-acc0-7a44b93eb806,TOPS,,data/img/f2244eb6-9735-4e0b-acc0-7a44b93eb806.png,Fabric: Mid-weight brushed knit. Stripe pattern. Ribbed edges. Crew neck. Long sleeves with dropped shoulders. Shell: 50% mohair/32% polyamide/18% wool. Hand wash. Made in Italy. Measurements: Measurements from size M Length: 25.5in / 65cm Sleeve: 35in / 89cm Chest: 49.5in / 126cm,733.917700,local_es
7,cat jacquard-motif sweater,GIVENCHY,6d8b2e0c-a469-4eb1-8243-51cc0513130e,TOPS,,data/img/6d8b2e0c-a469-4eb1-8243-51cc0513130e.png,timberwolf grey mohair blend knitted construction cat jacquard motif ribbed crew neck long sleeves unlined straight hem,733.669700,local_es
8,panelled sweater,BOSS,f29db84d-2d5d-49e2-aa48-632b3a0aed27,TOPS,,data/img/f29db84d-2d5d-49e2-aa48-632b3a0aed27.png,navy blue cotton blend panelled design fleece texture knitted construction ribbed knit trim crew neck long raglan sleeves mini logo tag unlined straight hem,733.619750,local_es
9,Drany Logo OverWool & Mohair Blend Crewneck Sweater at Nordstrom,ISABEL MARANT,e89b31b0-6435-4812-96fb-495fee6f98ec,TOPS,,data/img/e89b31b0-6435-4812-96fb-495fee6f98ec.png,"Dropped shoulder seams emphasize the boxy, slouchy fit of a lightweight sweater made from wool and mohair. Crewneck Long sleeves Ribbed cuffs and hem 50% wool, 32% mohair, 18% nylon Dry clean Imported Designer Clothing",733.035600,local_es


In [7]:
print(json.dumps(local_es.last_rendered, indent=2))

{
  "query": {
    "bool": {
      "should": [
        {
          "bool": {
            "filter": [
              {
                "bool": {
                  "filter": [
                    {
                      "range": {
                        "min_price": {
                          "gte": 0
                        }
                      }
                    }
                  ]
                }
              },
              {
                "bool": {
                  "filter": [
                    {
                      "range": {
                        "max_price": {
                          "lte": 2147483647
                        }
                      }
                    }
                  ]
                }
              },
              {
                "bool": {
                  "should": [
                    {
                      "match_phrase": {
                        "category": "tops"
                      }
                    }
           

In [9]:
from local_llm_judge.compare import compare_env
from local_llm_judge.log_stdout import enable

enable('local_llm_judge')

# Dougs queries
queries = ["boomer sweater", "genz sweater", "cheap genz socks"]

results = compare_env(queries, "cached_stag", "local_es")



Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

2025-01-29 17:01:48,462 - local_llm_judge.compare - INFO - Loaded 3005 cached option pair evals
2025-01-29 17:01:48,462 - local_llm_judge.compare - INFO - Comparing 3 queries
2025-01-29 17:01:48,462 - local_llm_judge.compare - INFO - Processing Query: boomer sweater - Department: w
2025-01-29 17:02:08,406 - local_llm_judge.search_backend - INFO - Applying brand age tag: Boomer
2025-01-29 17:02:11,646 - local_llm_judge.compare - INFO - Computing uncached feature: both_ways_desc for boomer sweater - db87e2f2-2503-4923-9e00-e943d63ab1e7 vs 498d4a80-27d7-48db-a72b-2e35c733b0a5
2025-01-29 17:02:13,316 - local_llm_judge.compare - INFO - Computing uncached feature: both_ways_category for boomer sweater - db87e2f2-2503-4923-9e00-e943d63ab1e7 vs 498d4a80-27d7-48db-a72b-2e35c733b0a5
2025-01-29 17:02:13,769 - local_llm_judge.compare - INFO - Computing uncached feature: both_ways_captions for boomer sweater - db87e2f2-2503-4923-9e00-e943d63ab1e7 vs 498d4a80-27d7-48db-a72b-2e35c733b0a5
2025-01-29 1

In [10]:
import pandas as pd

def _url_to_img_tag(path):
    return f"""<img
    style="width: 100px; height: 100px; object-fit: contain;"
            src="{path}"/>"""


def styled(df: pd.DataFrame) -> pd.DataFrame:
    return df.style.format({'image_url_lhs': _url_to_img_tag,
                            'image_url_rhs': _url_to_img_tag})


styled(results[['query', 'pref_lhs', 'backend_lhs', 'name_lhs', 'brand_name_lhs', 'category_lhs', 'image_url_lhs',
                'name_rhs', 'brand_name_rhs', 'image_url_rhs', 'backend_rhs', 'pref_rhs']])

,query,pref_lhs,backend_lhs,name_lhs,brand_name_lhs,category_lhs,image_url_lhs,name_rhs,brand_name_rhs,image_url_rhs,backend_rhs,pref_rhs
0,boomer sweater,0.638298,stag,Mock Neck Fair Isle Merino Wool Sweater at Nordstrom,VINEYARD VINES,TOPS,,Trans Love Crewneck,Lingua Franca,,local_es,0.361702
1,boomer sweater,0.638298,stag,Share Fair Isle Turtleneck Sweater at Nordstrom,NOISY MAY,TOPS,,"""bloody fabulous"" Crewneck",Lingua Franca,,local_es,0.361702
2,boomer sweater,0.638298,stag,Bali Stripe OverRecycled Cotton Blend Sweater at Nordstrom,NOISY MAY,TOPS,,"""women rising"" Crewneck",Lingua Franca,,local_es,0.361702
3,boomer sweater,0.401786,stag,Art Print Cashmere Sweater at Nordstrom,R13,TOPS,,Classic Wool Crewneck Sweater at Nordstrom,JIL SANDER,,local_es,0.598214
4,boomer sweater,0.456140,stag,chunky-knit sweater,ROBERTO COLLINA,TOPS,,Kingston sweater,ISABEL MARANT,,local_es,0.543860
5,boomer sweater,0.136364,stag,lace-embroidered sweater,ERMANNO SCERVINO,TOPS,,Monili chain-detail sweater,BRUNELLO CUCINELLI,,local_es,0.863636
6,boomer sweater,0.267606,stag,"""filthy mouthed wife"" Crewneck",LINGUA FRANCA,TOPS,,tie-dye sweater,MISSONI,,local_es,0.732394
7,boomer sweater,0.470588,stag,Fair Isle Mock Neck Sweater at Nordstrom,CASLON(R),TOPS,,Irlanda sweater,'S MAX MARA,,local_es,0.529412
8,boomer sweater,0.514793,stag,Archive Avalon Fair Isle Crewneck Sweater at Nordstrom,MARINE LAYER,TOPS,,Bans off Crewneck,Lingua Franca,,local_es,0.485207
9,boomer sweater,0.494145,stag,Anaya OverCable Stitch Sweater at Nordstrom,PRINCESS POLLY,TOPS,,Picture This Stripe Sweater at Nordstrom,Sanctuary,,local_es,0.505855


In [9]:
results['pref_lhs'].sum(), results['pref_rhs'].sum(), 

(157.30402021232794, 140.69597978767203)

In [ ]:
for _, row in results[results['query'] == 'vegan leather jacket'].iterrows():
    if row['pref_lhs'] > 0.4 and row['pref_lhs'] < 0.6:
        continue
    print("***")
    print(f"{row['name_lhs']} vs {row['name_rhs']}")
    print(f"{row['pref_lhs']} vs {row['pref_rhs']}")
    print(f"{row['brand_name_lhs']} vs {row['brand_name_rhs']}")
    print(f"{row['category_lhs']} vs {row['category_rhs']}")
    print()
    print("***")
    print(f"Desc LHS {row['desc_lhs']}")
    print(f"Desc rHS {row['desc_rhs']}")


    print('both_ways_desc', row['both_ways_desc'])
    # print(
    print('both_ways_category', row['both_ways_category'])
    print('both_ways_captions', row['both_ways_captions'])
    print('both_ways_brand', row['both_ways_brand'])
    print('both_ways_all_fields', row['both_ways_all_fields'])
    print()
    print()


In [ ]:
results.columns